In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np 


In [ ]:
img_array= cv2.imread("datasets\\with_mask\\00000_Mask.jpg")

In [ ]:
plt.imshow(img_array)

In [ ]:

Datadirectory = "datasets/"
Classes = ["with_mask","without_mask"]
for category in Classes:
    path=os.path.join(Datadirectory,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        #blacktorgb = cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB)
        plt.imshow(cv2.cvtColor(img_array,cv2.COLOR_BGR2RGB))
        plt.show()
        break
    break


In [ ]:
img_size = 224 ##ImageNet = 224*224

new_array = cv2.resize(img_array,(img_size,img_size))
plt.imshow(cv2.cvtColor(new_array,cv2.COLOR_BGR2RGB))
plt.show()

Reading the images and converting all to array


In [ ]:
training_Data = []

def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadirectory,category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(img_size,img_size))
                training_Data.append([new_array,class_num])
            except Exception as e:
                pass


In [ ]:
create_training_Data()

In [ ]:
print(len(training_Data))

In [ ]:
import random

random.shuffle(training_Data)


In [ ]:
X= [] #data features
Y= [] #label

for features,lable in training_Data:
    X.append(features)
    Y.append(lable)

X = np.array(X).reshape(-1,img_size,img_size,3)

In [ ]:
X.shape

In [ ]:
#normalize the data
X = X/225.0

In [ ]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out= open("Y.pickle","wb")
pickle.dump(Y,pickle_out)
pickle_out.close()


In [ ]:
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("Y.pickle", "rb")
Y= pickle.load(pickle_in)

Deep learning model for training - Transfer learning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = tf.keras.applications.mobilenet.MobileNet() #pre trained model


In [ ]:
model.summary()

Transfer learning


In [ ]:
base_input = model.layers[0].input


In [ ]:
base_output = model.layers[-4].output

In [ ]:
Flat_layers = layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layers)
final_output= layers.Activation('sigmoid')(final_output)


In [ ]:
new_model = keras.Model(inputs = base_input,outputs= final_output)

In [ ]:
new_model.summary()

Settings for binary classification (Face mask / without mask) 

In [ ]:
new_model.compile(loss ="binary_crossentropy",optimizer="adam",metrics=["accuracy"])


In [ ]:
X=np.array(X)
Y=np.array(Y)

In [ ]:
new_model.fit(X,Y, epochs=1,validation_split=0.1)

In [ ]:
evaluation = new_model.evaluate(X, Y)
print("Loss:", evaluation[0])
print("Accuracy:", evaluation[1])

In [ ]:
new_model.save('face_mask_detection_model.h5')

In [ ]:
new_model = tf.keras.models.load_model('face_mask_detection_model.h5')


Checking the network for presiction

In [ ]:
frame = cv2.imread("00003_Mask.jpg")


In [ ]:
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
final_image = cv2.resize(frame,(224,224))
final_image = np.expand_dims(final_image,axis=0)
final_image = final_image/255.0

In [ ]:
final_image = final_image.astype('float32')


In [ ]:
print(final_image.dtype)


In [ ]:
Predictions = new_model.predict(final_image)

In [ ]:
Predictions

checking the network for unknown image


In [ ]:
frame = cv2.imread('00003_Mask.jpg')

In [ ]:
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

In [ ]:
gray.shape

In [ ]:
faces = faceCascade.detectMultiScale(gray,1.1,4)
for x,y,w,h in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if len(facess) == 0:
        print("Face not detected")
    else:
        for (ex,ey,ew,eh) in facess:
            face_roi = roi_color[ey:ey+eh, ex:ex + ew]

In [ ]:
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

In [ ]:
plt.imshow(cv2.cvtColor(face_roi,cv2.COLOR_BGR2RGB))

In [ ]:
final_image = cv2.resize(face_roi,(224,224))
final_image = np.expand_dims(final_image,axis=0)
final_image = final_image/255.0

In [ ]:

final_image = final_image.astype('float32')


In [ ]:
print(final_image.dtype)


In [ ]:
print(new_model.summary())




In [ ]:
Predictions = new_model.predict(final_image)

In [ ]:
Predictions

REALTIME VIDEO DEMO

FACE MASK DETECTION

In [ ]:
#YOUTUBE CODE

import cv2
path  = "haarcascade_frontalface_default.xml"
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

#set the rectangle background to white
rectangle_bar = (255,255,255)
#make a black image
img = np.zeros((500,500))
#set some text
text ="Some text in a box!"
#get the width and height of the text box
(text_width,text_height) = cv2.getTextSize(text,font,fontScale=font_scale,thickness=1)[0]
#set the text start position
text_offset_x = 10
text_offset_y = img.shape[0]-25
#make the coords of the box with a small padding of two pixels
box_coords = ((text_offset_x , text_offset_y),(text_offset_x + text_width + 2 , text_offset_y - text_height -2))
cv2.rectangle(img,box_coords[0],box_coords[1],rectangle_bar, cv2.FILLED)
cv2.putText(img,text,(text_offset_x,text_offset_y),font,fontScale=font_scale,color=(0,0,0),thickness=1)

cap= cv2.VideoCapture(1)
#check if the web cam is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")
while True:
    ret,frame=cap.read()
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #print (faceCascade.empty())
    faces = faceCascade.detectMultiScale(gray,1.1,4)
    for x,y,w,h in faces:
        roi_gray = gray[y:y+h , x:x+w]
        roi_color = frame[y:y+h , x:x+w]
        cv2.rectangle(frame,(x,y),(x+w, y+h),(255,0,0),2)
        facess = faceCascade.detectMultiScale(roi_gray)
        if len(facess) ==0:
            print("Face not detected")
        else:
            for (ex,ey,ew,eh) in facess:
                face_roi = roi_color[ey+eh,ex:ex+ew]
    final_image = cv2.resize(face_roi,(224,224))
    final_image = np.expand_dims(final_image,axis=0)
    final_image=final_image/255.0
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    Predictions = new_model.predict(final_image)

    font_scale = 1.5
    font = cv2.FONT_HERSHEY_PLAIN


    if (Predictions [0][0] > 0.5):
        status = "No Mask"

        x1,y1,w1,h1 = 0,0,175,75
        #draw black background rectangle
        cv2.rectangle(frame,(x1,x1),(x1 + w1, y1+h1),(0,0,0),-1)
        #add text
        cv2.putText(frame,status,(x1 + int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)

        cv2.putText(frame, status (100,150),font,3,(0,0,255),2,cv2.LINE_4)

        cv2.rectangle(frame,(x,y),(x+w, y+h),(0,0,255))
    else:
        status ="Face Mask"

        x1,y1,w1,h1 = 0,0,175,75
        #draw black background rectangle
        cv2.rectangle(frame,(x1 + int(w1/10),y1 + int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX,0.7 , (0,255,0),2)
        cv2.putText(frame, status (100,150),font,3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w, y+h),(0,255,0))

        cv2.imshow('Face mask detection tutotial',frame)
        if cv2.waitKey(2)& 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
path  = "haarcascade_frontalface_default.xml"
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

import numpy as np
import tensorflow as tf

# ... (Previous code)

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    
    for x, y, w, h in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        facess = faceCascade.detectMultiScale(roi_gray)
        
        if len(facess) == 0:
            print("Face not detected")
        else:
            for (ex, ey, ew, eh) in facess:
                face_roi = roi_color[ey:ey+eh, ex:ex+ew]
    
    final_image = cv2.resize(face_roi, (224, 224))
    final_image = np.expand_dims(final_image, axis=0)
    final_image = final_image / 255.0
    
    Predictions = new_model.predict(final_image)
    
    font_scale = 1.5
    font = cv2.FONT_HERSHEY_PLAIN
    
    if Predictions  [0][0] > 0.5:
        status = "No Mask"
        x1, y1, w1, h1 = 0, 0, 175,75
        cv2.rectangle(frame, (x1, y1), (x1 + w1, y1+h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1 + int(w1/10), y1+int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))
    else:
        status = "Face Mask"
        x1, y1, w1, h1 = 0, 0, 175,75
        cv2.rectangle(frame, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0))

    cv2.imshow('Face mask detection ', frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the face detection Haar Cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the mask detection model
mask_detection_model = tf.keras.models.load_model('face_mask_detection_model.h5')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face_roi = frame[y:y + h, x:x + w]

        # Preprocess the face for mask detection
        face_roi = cv2.resize(face_roi, (224, 224))
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0

        # Make mask detection predictions
        mask_predictions = mask_detection_model.predict(face_roi)

        # Display the face rectangle with label
        if mask_predictions[0][0] > 0.5:
            label = "No Mask"
            color = (0, 0, 255)  # Red for no mask
        else:
            label = "Mask"
            color = (0, 255, 0)  # Green for mask

        # Draw the face rectangle and label
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Display the frame with face and mask detection
    cv2.imshow('Face and Mask Detection', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
